# Classificateur d'images simple

Ce notebook vous montre comment classifier des images en utilisant un réseau neuronal pré-entraîné.

**Ce que vous allez apprendre :**
- Comment charger et utiliser un modèle pré-entraîné
- Prétraitement des images
- Faire des prédictions sur des images
- Comprendre les scores de confiance

**Cas d'utilisation :** Identifier des objets dans des images (comme "chat", "chien", "voiture", etc.)

---


## Étape 1 : Importer les bibliothèques nécessaires

Importons les outils dont nous avons besoin. Ne vous inquiétez pas si vous ne comprenez pas encore tout cela !


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Étape 2 : Charger le modèle pré-entraîné

Nous allons utiliser **MobileNetV2**, un réseau neuronal déjà entraîné sur des millions d'images.

C'est ce qu'on appelle **l'apprentissage par transfert** - utiliser un modèle entraîné par quelqu'un d'autre !


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Étape 3 : Fonctions utilitaires

Créons des fonctions pour charger et préparer les images pour notre modèle.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Étape 4 : Tester sur des images d'exemple

Essayons de classifier quelques images trouvées sur Internet !


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Classifier chaque image


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Étape 5 : Essayez avec vos propres images !

Remplacez l'URL ci-dessous par n'importe quelle URL d'image que vous souhaitez classifier.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Que s'est-il passé ?

1. **Nous avons chargé un modèle pré-entraîné** - MobileNetV2 a été entraîné sur des millions d'images  
2. **Nous avons prétraité les images** - Redimensionnées et formatées pour le modèle  
3. **Le modèle a fait des prédictions** - Il a produit des probabilités pour 1000 classes d'objets  
4. **Nous avons décodé les résultats** - Converti les chiffres en étiquettes compréhensibles  

### Comprendre les scores de confiance

- **90-100 %** : Très confiant (presque certainement correct)  
- **70-90 %** : Confiant (probablement correct)  
- **50-70 %** : Moyennement confiant (peut être correct)  
- **Moins de 50 %** : Peu confiant (incertain)  

### Pourquoi les prédictions pourraient être erronées ?

- **Angle ou éclairage inhabituel** - Le modèle a été entraîné sur des photos classiques  
- **Objets multiples** - Le modèle s'attend à un objet principal  
- **Objets rares** - Le modèle ne connaît que 1000 catégories  
- **Image de mauvaise qualité** - Les images floues ou pixelisées sont plus difficiles à analyser  

---


## 🚀 Prochaines étapes

1. **Essayez différentes images :**
   - Trouvez des images sur [Unsplash](https://unsplash.com)
   - Clic droit → "Copier l'adresse de l'image" pour obtenir l'URL

2. **Expérimentez :**
   - Que se passe-t-il avec l'art abstrait ?
   - Peut-il reconnaître des objets sous différents angles ?
   - Comment gère-t-il plusieurs objets ?

3. **Apprenez-en davantage :**
   - Explorez les [leçons sur la vision par ordinateur](../lessons/4-ComputerVision/README.md)
   - Apprenez à entraîner votre propre classificateur d'images
   - Comprenez le fonctionnement des CNNs (Convolutional Neural Networks)

---

## 🎉 Félicitations !

Vous venez de créer un classificateur d'images en utilisant un réseau neuronal de pointe !

Cette même technique alimente :
- Google Photos (organisation de vos photos)
- Voitures autonomes (reconnaissance d'objets)
- Diagnostic médical (analyse des radiographies)
- Contrôle qualité (détection de défauts)

Continuez à explorer et à apprendre ! 🚀



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction humaine professionnelle. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
